# BASIC RAG - Evaluation
##  Trial of metrics 

In [35]:
import warnings
warnings.filterwarnings('ignore')

In [36]:
import nest_asyncio
nest_asyncio.apply()

On charge les questions GAIA

In [37]:
import csv 

questions_set = []
format_set = []
with open("../GAIA_questions.csv", 'r', newline='', encoding='utf-8') as file :
    csv_lector = csv.DictReader(file, delimiter=';')
    for line in csv_lector : 
        questions_set.append({"query": line["Question"], "response" : f"response : {line['Answer']}", "GroundTruth" : line['Groundtruth']})
        format_set.append(line["Format"])
print(questions_set)

[{'query': 'Quel est le pourcentage les combustibles fossiles sont responsables des emissions mondiales de gaz a effet de serre? ', 'response': 'response : 75', 'GroundTruth': 'T '}, {'query': 'Quel est le pourcentage les combustibles fossiles sont responsables des emissions de dioxyde de carabone? ', 'response': 'response : 90', 'GroundTruth': 'T '}, {'query': "Combien de millions d'hectares de forets sont detruite chaque annee a cause de la deforestation ?", 'response': 'response : 12', 'GroundTruth': 'T '}, {'query': 'Combien de millions de personnes meurent chaque annee a cause de facters environnementaux? ', 'response': 'response : 13', 'GroundTruth': 'T '}, {'query': 'En moyenne, combien de millions de personnes ont ete deplaces en raison de phenomenes meterologiques chaque annee lors de la derniere decennie ?  ecris millions sans abreviation', 'response': 'response : 23,1', 'GroundTruth': 'T '}, {'query': "De combien de degre de difference etait la temperature moyenne de 2019 oa

On récupère la database chromaDB

In [38]:
import chromadb
from chromadb.utils import embedding_functions

with open("log/embedding_function.txt", 'r') as file : 
    embedding_function = eval(file.read())

chroma_client = chromadb.PersistentClient("./dbs/documentation/chroma_index")
chroma_collection = chroma_client.get_collection("Basic_rag", embedding_function=embedding_function)

In [39]:
from trulens_eval import Tru 

tru = Tru()
tru.reset_database()

In [40]:
from trulens_eval import Feedback, Select
from trulens_eval.feedback import Groundedness, GroundTruthAgreement
from trulens_eval.feedback.provider.openai import OpenAI as fOpenAI

import numpy as np

# Initialize provider class
fopenai = fOpenAI()

grounded = Groundedness(groundedness_provider=fopenai)

# Define a groundedness feedback function
f_groundedness = (
    Feedback(grounded.groundedness_measure_with_cot_reasons, name = "Groundedness")
    .on(Select.RecordCalls.retrieve.rets.collect())
    .on_output()
    .aggregate(grounded.grounded_statements_aggregator) # List[Dict] : a list of scores. Each list index is a contexte 
) 

# f_groundedness = (
#     Feedback(grounded.groundedness_measure_with_cot_reasons, name = "Groundedness")
#     .on(Select.Record.app.combine_documents_chain._call.args.inputs.input_documents[:].page_content)
#     .on_output()
#     .aggregate(grounded.grounded_statements_aggregator) # List[Dict] : a list of scores. Each list index is a contexte 
# ) 



# Question/answer relevance between overall question and answer.
f_qa_relevance = (
    Feedback(fopenai.relevance_with_cot_reasons, name = "Answer Relevance")
    .on(Select.RecordCalls.retrieve.args.query)
    .on_output()
)

# Question/statement relevance between question and each context chunk.
f_context_relevance = (
    Feedback(fopenai.qs_relevance_with_cot_reasons, name = "Context Relevance")
    .on(Select.RecordCalls.retrieve.args.query)
    .on(Select.RecordCalls.retrieve.rets.collect())
    .aggregate(np.mean)
)


f_groundtruth = (
    Feedback(GroundTruthAgreement(questions_set).agreement_measure, name = "Ground Truth")
    .on_input_output()
    )

f_bert_score = (
    Feedback(GroundTruthAgreement(questions_set).bert_score, name="Bert score").on_input_output())

f_bleu = (
    Feedback(GroundTruthAgreement(questions_set).bleu, name="Bleu score").on_input_output()
)

f_agreement_measure =(
    Feedback(GroundTruthAgreement(questions_set).agreement_measure, name="Agreement mesure").on_input_output()
)


✅ In Groundedness, input source will be set to __record__.app.retrieve.rets.collect() .
✅ In Groundedness, input statement will be set to __record__.main_output or `Select.RecordOutput` .
✅ In Answer Relevance, input prompt will be set to __record__.app.retrieve.args.query .
✅ In Answer Relevance, input response will be set to __record__.main_output or `Select.RecordOutput` .
✅ In Context Relevance, input question will be set to __record__.app.retrieve.args.query .
✅ In Context Relevance, input statement will be set to __record__.app.retrieve.rets.collect() .
✅ In Ground Truth, input prompt will be set to __record__.main_input or `Select.RecordInput` .
✅ In Ground Truth, input response will be set to __record__.main_output or `Select.RecordOutput` .
✅ In Bert score, input prompt will be set to __record__.main_input or `Select.RecordInput` .
✅ In Bert score, input response will be set to __record__.main_output or `Select.RecordOutput` .
✅ In Bleu score, input prompt will be set to __rec

In [44]:
from openai import OpenAI
from trulens_eval.tru_custom_app import instrument 

oai_client = OpenAI()

def my_retrieve(query):
    results = chroma_collection.query(
            query_texts=query,
            n_results=2
        )
    # for i in range(len(results["documents"][0])):
    #     results["documents"][0][i].replace("’", "'")
    #     results["documents"][0][i].replace('\n', '') 
    # print(results["documents"][0])
    return results['documents']

def my_completion(docs, query):
    context = ""
    for ref in docs[0] :
        context += f"{ref}\n------------------------\n"
    completion = oai_client.chat.completions.create(
        model = "gpt-3.5-turbo",
        temperature= 0,
        messages=[
            {"role" : "system", "content" : f"Tu es un assistant IA. Lorsqu'on te pose une question, écris toujours 'response :' puis la réponse dans le format {format_set.pop()}. \nExemple : Quelle est l'annee de la revolution francaise? avec le format annee - 'XXXX', cela donne : 'response : 1789'\nExemple : Combien de pourcentage d'eau notre corps est constitué? avec le format nombre décimal 'XX,X', cela donne : 'response : 65,1' "},
            {"role": "user", "content": f"Nous avons le ou les contenus fournis ci dessous. \n------------------------\n{context}\nEn te basant sur le contexte donné, réponds à cette question, et respecte le format indiqué : {query}\nRéponse : "}
        ]
    ).choices[0].message.content 
    return completion

    


class RAG_APP:
    @instrument
    def retrieve(self, query):
        return my_retrieve(query)

    @instrument 
    def completion(self, docs, query):
        return my_completion(docs, query)
    
    @instrument
    def query(self, query):
        context = self.retrieve(query)
        completion = self.completion(context, query)
        return completion

rag_app = RAG_APP()

In [47]:
from trulens_eval import TruCustomApp

format_set = []
with open("../GAIA_questions.csv", 'r', newline='', encoding='utf-8') as file :
    csv_lector = csv.DictReader(file, delimiter=';')
    for line in csv_lector : 
        format_set.append(line["Format"])




tru_app = TruCustomApp(rag_app, app_id = "Basic RAG", feedbacks = [f_groundtruth])
with tru_app as recording :
    i = 0
    for question in questions_set :
        if question["GroundTruth"] == 'T' :
            rep = rag_app.query(question["query"])
            print(f"{i:>2} - {question['response']:<30} | {rep}")
            i += 1

tru_app = TruCustomApp(rag_app, app_id = "Basic RAG", feedbacks = [f_qa_relevance, f_context_relevance, f_groundedness])
with tru_app as recording :
    i = 0
    for question in questions_set :
        if question["GroundTruth"] == 'F' :
            rep = rag_app.query(question["query"])
            print(f"{i:>2} - {rep}")
            i += 1

Function <function RAG_APP.query at 0x0000015B47901C10> was not found during instrumentation walk. Make sure it is accessible by traversing app <__main__.RAG_APP object at 0x0000015B4AA74160> or provide a bound method for it as TruCustomApp constructor argument `methods_to_instrument`.
Function <function RAG_APP.completion at 0x0000015B4AAB1C10> was not found during instrumentation walk. Make sure it is accessible by traversing app <__main__.RAG_APP object at 0x0000015B4AA74160> or provide a bound method for it as TruCustomApp constructor argument `methods_to_instrument`.
Function <function RAG_APP.query at 0x0000015B4AAB1B80> was not found during instrumentation walk. Make sure it is accessible by traversing app <__main__.RAG_APP object at 0x0000015B4AA74160> or provide a bound method for it as TruCustomApp constructor argument `methods_to_instrument`.
Function <function RAG_APP.query at 0x0000015B4AA65670> was not found during instrumentation walk. Make sure it is accessible by trave

 0 - response : 48                  | response : La question ne fournit pas d'informations précises sur la concentration de dioxyde de carbone (CO2) dans l'atmosphère en 2020 par rapport à son niveau préindustriel.
 1 - response : 1988                | Depuis quelle annee un groupe d'experts sur l'evolution du climat (GIEC) travaille-t-il ensemble ?
Réponse : Le GIEC travaille ensemble depuis 1988.
 2 - response : 30                  | response : L'intervalle de temps généralement utilisé comme référence pour calculer le climat est de 1990 à 2018.
 3 - response : 3                   | response : Aucun facteur principal n'est mentionné comme contribuant au climat de la Terre dans le texte.
 4 - response : 66,5                | response : Environ 66,5 millions d'enfants sont estimés être touchés chaque année par des catastrophes naturelles.
 5 - response : 175                 | response : Environ 175 millions d'enfants seront estimés être touchés chaque année par des catastrophes dues au

Function <function RAG_APP.query at 0x0000015B47901C10> was not found during instrumentation walk. Make sure it is accessible by traversing app <__main__.RAG_APP object at 0x0000015B4AA74160> or provide a bound method for it as TruCustomApp constructor argument `methods_to_instrument`.
Function <function RAG_APP.completion at 0x0000015B4AAB1C10> was not found during instrumentation walk. Make sure it is accessible by traversing app <__main__.RAG_APP object at 0x0000015B4AA74160> or provide a bound method for it as TruCustomApp constructor argument `methods_to_instrument`.
Function <function RAG_APP.query at 0x0000015B4AAB1B80> was not found during instrumentation walk. Make sure it is accessible by traversing app <__main__.RAG_APP object at 0x0000015B4AA74160> or provide a bound method for it as TruCustomApp constructor argument `methods_to_instrument`.
Function <function RAG_APP.query at 0x0000015B4AA65670> was not found during instrumentation walk. Make sure it is accessible by trave

35 - response : 13                  | response : Les émissions nettes des terres boisées, notamment à cause de la déforestation, représentent environ 13 % des émissions anthropiques annuelles de GES dans le monde.
 0 - response : Les principaux contributeurs au changement climatique mondial sont les activités humaines telles que la combustion des combustibles fossiles (charbon, pétrole, gaz naturel), la déforestation, l'agriculture intensive et l'industrie. Ces activités entraînent une augmentation des émissions de gaz à effet de serre, tels que le dioxyde de carbone (CO2), le méthane (CH4) et le protoxyde d'azote (N2O), qui contribuent au réchauffement de la planète.
 1 - response : Les combustibles fossiles, tels que le charbon, le pétrole et le gaz, sont l'une des principales sources d'émissions de gaz à effet de serre dans le monde. Cependant, le pourcentage précis des émissions attribuées à ces combustibles n'est pas mentionné dans le texte fourni.
 2 - response : La production d'

In [48]:
tru.get_leaderboard(app_ids=[tru_app.app_id])

,Ground Truth,Groundedness,Context Relevance,Answer Relevance,latency,total_cost
app_id,,,,,,
Basic RAG,0.682979,0.698413,0.811905,0.988095,3.067416,0.000841


In [1]:
tru.run_dashboard() 

NameError: name 'tru' is not defined